In [25]:

import torch
import math
from utils.aihub_kpop_dataset import KpopImageDatasetwT
from torch.utils.data import DataLoader
from model import PoseHRNet
from torch import nn, Tensor
from torch.utils.tensorboard import SummaryWriter
import tqdm
from tqdm import tqdm_notebook

writer = SummaryWriter()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device ? : {}'.format(device))
mymodel = PoseHRNet('w32', num_joints=29)
mydataset = KpopImageDatasetwT()
mydataloader = DataLoader(dataset=mydataset, batch_size=128, shuffle=True)
myloss = nn.MSELoss()
myoptimizer = torch.optim.Adam(mymodel.parameters(), lr=1e-3)


device ? : cpu


In [26]:
def train(dataloader, model, loss_fn, optimizer, save_path):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(tqdm.tqdm(dataloader)):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 32 == 0:
            loss, current = loss.item(), batch * len(X)
            writer.add_scalar("Loss/5Batches", loss, batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    # after train
    if save_path is None:
        save_path = 'latest.pth'
    torch.save(model, save_path)
    # save entire model
    

In [27]:
EPOCHS = 5
SAVE_PATH = None

for t in tqdm.tqdm(range(EPOCHS)):
    print(f"Epoch {t+1}\n-------------------------------")
    train(mydataloader, mymodel, myloss, myoptimizer, save_path=SAVE_PATH)
    # test(mydataloader, model, myloss)
print("Done!")

writer.flush()

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1
-------------------------------


loss: 120.144257  [    0/ 3447]
